In [1]:
#Can we add:

##number of unique different products bought
##main product bought both monetary and amount
##Clean up Average Days so if it is N/A because there was only one purchase then we calculate how many days since that purchase to end of our data.
##could we also see Number of Days Purchases were made?
#We have the cohort analysis that shows a lot of retention for the first two months. Let's label these clients that last to the end of the period as long term. I can explain in more detail.. Matt Retention Cohort.ipynb
##column simply stating if the customer went through a refund (1,0)
##Main Channel used for purchases. (Monetary)
#?This might be hard but was there a shift in channel used? For instance if went from phone to online have P -> O or something.
##Most Recent channel used?
##Can also create three general groups based off the score of the RFM. I have the code for this and could use your help to work out why my clustering doesn't work as i want to check what would be the ideal amount of k means clusters would be.


In [2]:
#df_customer['RFM_Score'] = df_customer['Recency_Quantitle'] + df_customer['Frequency_Quantitle'] + df_customer['Monetary_Value_Quantitle']

In [3]:
#def segments(df):
#    if df['RFM_Score'] > 10 :
#        return 'Gold'
#    elif (df['RFM_Score'] > 5) and (df['RFM_Score'] <= 10 ):
#        return 'Silver'
#    else:
#        return 'Bronze'
#df_customer['General_Segment'] = df_customer.apply(segments,axis=1)
#df_customer.groupby('General_Segment').agg({'Recency':'mean','Frequency':'mean',
                                   # 'Monetary_Value':['mean','count']}).round(1)

In [4]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df = pd.read_parquet('/content/drive/MyDrive/Elective_Groups/Capstone/Step_2_transactions_dataset_outlier_removal.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26061261 entries, 0 to 26061260
Data columns (total 8 columns):
 #   Column         Dtype         
---  ------         -----         
 0   date_order     datetime64[ns]
 1   product_id     category      
 2   client_id      category      
 3   sales_net      float32       
 4   quantity       int32         
 5   order_channel  category      
 6   branch_id      category      
 7   month          object        
dtypes: category(4), datetime64[ns](1), float32(1), int32(1), object(1)
memory usage: 896.0+ MB


In [7]:
df.head()

,date_order,product_id,client_id,sales_net,quantity,order_channel,branch_id,month
0,2017-09-25,2376333,188502,155.443207,3,online,2732,2017-09
1,2017-09-25,2536756,1977896,365.766235,21,online,6184,2017-09
2,2017-09-25,203377,2086861,1093.374023,3,by phone,7051,2017-09
3,2017-09-25,745693,482822,1.449000,5,by phone,2771,2017-09
4,2017-09-25,683924,224223,12.033600,3,at the store,9463,2017-09


1.1 Build Up Customer Table

In [8]:
df_customer = pd.DataFrame()

In [9]:
df_customer['client_id'] = df['client_id'].unique()
df_customer.head()

,client_id
0,188502
1,1977896
2,2086861
3,482822
4,224223


1.2 How many times the client bought

In [10]:
amount_track = df.groupby('client_id').agg({'quantity':['sum','mean'],'sales_net': ['count','sum','mean']}).reset_index()
amount_track.columns = list(map(''.join, amount_track.columns.values))
amount_track = amount_track.rename(
    columns={"quantitysum": "Total_Quantity", 
             "sales_netcount": "Number_of_Purchase", 
             "sales_netsum": "Total_Sales", 
             "quantitymean": "Average_Quantity", 
             "sales_netmean": "Average_Sales"})
amount_track

,client_id,Total_Quantity,Average_Quantity,Number_of_Purchase,Total_Sales,Average_Sales
0,1000078,13361,86.200000,155,30274.775391,195.321136
1,1000352,24795,44.197861,561,78321.203125,139.609985
2,1000380,0,NaN,0,0.000000,NaN
3,1000618,0,NaN,0,0.000000,NaN
4,1000680,0,NaN,0,0.000000,NaN
...,...,...,...,...,...,...
170584,701328,0,NaN,0,0.000000,NaN
170585,742185,44,11.000000,4,0.000000,0.000000
170586,751654,0,NaN,0,0.000000,NaN
170587,878141,0,NaN,0,0.000000,NaN


In [11]:
df_customer = df_customer.merge(amount_track,on='client_id')

1.3 Refund

In [12]:
refund_track = df.loc[df.sales_net < 0].groupby('client_id').agg({'quantity':['sum','mean'],'sales_net': ['count','sum','mean']}).reset_index()
refund_track.columns = list(map(''.join, refund_track.columns.values))
refund_track = refund_track.rename(
    columns={"quantitysum": "R_Total_Quantity", 
             "sales_netcount": "R_Number_of_Purchase", 
             "sales_netsum": "R_Total_Sales", 
             "quantitymean": "R_Average_Quantity", 
             "sales_netmean": "R_Average_Sales"})
refund_track

,client_id,R_Total_Quantity,R_Average_Quantity,R_Number_of_Purchase,R_Total_Sales,R_Average_Sales
0,1000078,3235,82.948718,39,-9034.760742,-231.660538
1,1000352,450,75.000000,6,-1809.559814,-301.593292
2,1000380,0,NaN,0,0.000000,NaN
3,1000618,0,NaN,0,0.000000,NaN
4,1000680,0,NaN,0,0.000000,NaN
...,...,...,...,...,...,...
170584,701328,0,NaN,0,0.000000,NaN
170585,742185,22,11.000000,2,-129.070480,-64.535240
170586,751654,0,NaN,0,0.000000,NaN
170587,878141,0,NaN,0,0.000000,NaN


In [13]:
df_customer = df_customer.merge(refund_track,on='client_id')

1.4 Last Transaction

In [14]:
date_track = df.groupby('client_id').agg({'date_order':['max','min']}).reset_index()
date_track.columns = list(map(''.join, date_track.columns.values))
date_track = date_track.rename(
    columns={"date_ordermax": "Last_Activity_Day", 
             "date_ordermin": "First_Activity_Day"})
date_track['Last_Activity_Month'] = date_track['Last_Activity_Day'].dt.strftime('%Y-%m')
date_track['First_Activity_Month'] = date_track['First_Activity_Day'].dt.strftime('%Y-%m')
date_track

,client_id,Last_Activity_Day,First_Activity_Day,Last_Activity_Month,First_Activity_Month
0,1000078,2019-03-22,2017-09-27,2019-03,2017-09
1,1000352,2019-09-06,2017-09-26,2019-09,2017-09
2,1000380,NaT,NaT,NaN,NaN
3,1000618,NaT,NaT,NaN,NaN
4,1000680,NaT,NaT,NaN,NaN
...,...,...,...,...,...
170584,701328,NaT,NaT,NaN,NaN
170585,742185,2019-06-26,2019-06-26,2019-06,2019-06
170586,751654,NaT,NaT,NaN,NaN
170587,878141,NaT,NaT,NaN,NaN


In [15]:
df_customer = df_customer.merge(date_track,on='client_id')
df_customer

,client_id,Total_Quantity,Average_Quantity,Number_of_Purchase,Total_Sales,Average_Sales,R_Total_Quantity,R_Average_Quantity,R_Number_of_Purchase,R_Total_Sales,R_Average_Sales,Last_Activity_Day,First_Activity_Day,Last_Activity_Month,First_Activity_Month
0,188502,140138,227.496753,616,66700.335938,108.279770,42,5.250000,8,-1054.278564,-131.784821,2019-09-05,2017-09-25,2019-09,2017-09
1,1977896,293627,567.943907,517,145390.906250,281.220337,260,43.333333,6,-2302.838867,-383.806488,2018-11-05,2017-09-25,2018-11,2017-09
2,2086861,19161,58.956923,325,40332.011719,124.098495,2018,504.500000,4,-1377.861328,-344.465332,2019-09-12,2017-09-25,2019-09,2017-09
3,482822,6740,37.444444,180,16716.583984,92.869911,0,NaN,0,0.000000,NaN,2019-09-06,2017-09-25,2019-09,2017-09
4,224223,27219,26.096836,1043,124999.437500,119.846054,105,8.076923,13,-5282.967285,-406.382111,2019-09-20,2017-09-25,2019-09,2017-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148720,144085,69,13.800000,5,235.405930,47.081184,21,21.000000,1,-1232.886475,-1232.886475,2019-08-29,2019-08-27,2019-08,2019-08
148721,450600,18,3.000000,6,0.000000,0.000000,3,3.000000,1,-2.290800,-2.290800,2019-08-06,2018-10-30,2019-08,2018-10
148722,1511299,132,11.000000,12,0.000000,0.000000,66,11.000000,6,-1549.754761,-258.292450,2019-07-22,2019-07-22,2019-07,2019-07
148723,923579,48,4.000000,12,2428.615967,202.384674,0,NaN,0,0.000000,NaN,2019-07-02,2019-07-02,2019-07,2019-07


1.5 Average number of days between transactions

In [16]:
date_btw = df[['client_id','date_order']].sort_values(['client_id','date_order'])
date_btw[['client_id_shift','date_order_shift']] = date_btw.shift()

In [17]:
date_btw['date_diff'] = (date_btw['date_order'] - date_btw['date_order_shift']) / np.timedelta64(1, 'D')
date_diff = date_btw.loc[date_btw['client_id'] == date_btw['client_id_shift']]
date_diff

,client_id,date_order,client_id_shift,date_order_shift,date_diff
56062,1000078,2017-09-27,1000078,2017-09-27,0.0
588701,1000078,2017-09-27,1000078,2017-09-27,0.0
1050277,1000078,2017-09-27,1000078,2017-09-27,0.0
1998860,1000078,2017-11-02,1000078,2017-09-27,36.0
2020764,1000078,2017-11-02,1000078,2017-11-02,0.0
...,...,...,...,...,...
26054096,923579,2019-07-02,923579,2019-07-02,0.0
26055591,923579,2019-07-02,923579,2019-07-02,0.0
26055817,923579,2019-07-02,923579,2019-07-02,0.0
26057524,923579,2019-07-02,923579,2019-07-02,0.0


In [18]:
date_diff_group = date_diff.groupby('client_id').agg({'date_diff':['mean']}).reset_index()
date_diff_group.columns = list(map(''.join, date_diff_group.columns.values))
date_diff_group = date_diff_group.rename(
    columns={"date_diffmean": "Average_Purchase_Day_All"})

In [19]:
df_customer = df_customer.merge(date_diff_group, on='client_id')
df_customer

,client_id,Total_Quantity,Average_Quantity,Number_of_Purchase,Total_Sales,Average_Sales,R_Total_Quantity,R_Average_Quantity,R_Number_of_Purchase,R_Total_Sales,R_Average_Sales,Last_Activity_Day,First_Activity_Day,Last_Activity_Month,First_Activity_Month,Average_Purchase_Day_All
0,188502,140138,227.496753,616,66700.335938,108.279770,42,5.250000,8,-1054.278564,-131.784821,2019-09-05,2017-09-25,2019-09,2017-09,1.154472
1,1977896,293627,567.943907,517,145390.906250,281.220337,260,43.333333,6,-2302.838867,-383.806488,2018-11-05,2017-09-25,2018-11,2017-09,0.786822
2,2086861,19161,58.956923,325,40332.011719,124.098495,2018,504.500000,4,-1377.861328,-344.465332,2019-09-12,2017-09-25,2019-09,2017-09,2.212963
3,482822,6740,37.444444,180,16716.583984,92.869911,0,NaN,0,0.000000,NaN,2019-09-06,2017-09-25,2019-09,2017-09,3.972067
4,224223,27219,26.096836,1043,124999.437500,119.846054,105,8.076923,13,-5282.967285,-406.382111,2019-09-20,2017-09-25,2019-09,2017-09,0.695777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148720,144085,69,13.800000,5,235.405930,47.081184,21,21.000000,1,-1232.886475,-1232.886475,2019-08-29,2019-08-27,2019-08,2019-08,0.500000
148721,450600,18,3.000000,6,0.000000,0.000000,3,3.000000,1,-2.290800,-2.290800,2019-08-06,2018-10-30,2019-08,2018-10,56.000000
148722,1511299,132,11.000000,12,0.000000,0.000000,66,11.000000,6,-1549.754761,-258.292450,2019-07-22,2019-07-22,2019-07,2019-07,0.000000
148723,923579,48,4.000000,12,2428.615967,202.384674,0,NaN,0,0.000000,NaN,2019-07-02,2019-07-02,2019-07,2019-07,0.000000


1.5.2 Average number of unique days between transactions

In [20]:
date_diff_group = date_diff.loc[date_diff.date_diff > 0].groupby('client_id').agg({'date_diff':['mean']}).reset_index()
date_diff_group.columns = list(map(''.join, date_diff_group.columns.values))
date_diff_group = date_diff_group.rename(
    columns={"date_diffmean": "Average_Purchase_Day"})

In [21]:
df_customer = df_customer.merge(date_diff_group, on='client_id')
df_customer

,client_id,Total_Quantity,Average_Quantity,Number_of_Purchase,Total_Sales,Average_Sales,R_Total_Quantity,R_Average_Quantity,R_Number_of_Purchase,R_Total_Sales,R_Average_Sales,Last_Activity_Day,First_Activity_Day,Last_Activity_Month,First_Activity_Month,Average_Purchase_Day_All,Average_Purchase_Day
0,188502,140138,227.496753,616,66700.335938,108.279770,42,5.250000,8,-1054.278564,-131.784821,2019-09-05,2017-09-25,2019-09,2017-09,1.154472,5.819672
1,1977896,293627,567.943907,517,145390.906250,281.220337,260,43.333333,6,-2302.838867,-383.806488,2018-11-05,2017-09-25,2018-11,2017-09,0.786822,4.951220
2,2086861,19161,58.956923,325,40332.011719,124.098495,2018,504.500000,4,-1377.861328,-344.465332,2019-09-12,2017-09-25,2019-09,2017-09,2.212963,6.894231
3,482822,6740,37.444444,180,16716.583984,92.869911,0,NaN,0,0.000000,NaN,2019-09-06,2017-09-25,2019-09,2017-09,3.972067,16.534884
4,224223,27219,26.096836,1043,124999.437500,119.846054,105,8.076923,13,-5282.967285,-406.382111,2019-09-20,2017-09-25,2019-09,2017-09,0.695777,3.625000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148720,144085,69,13.800000,5,235.405930,47.081184,21,21.000000,1,-1232.886475,-1232.886475,2019-08-29,2019-08-27,2019-08,2019-08,0.500000,2.000000
148721,450600,18,3.000000,6,0.000000,0.000000,3,3.000000,1,-2.290800,-2.290800,2019-08-06,2018-10-30,2019-08,2018-10,56.000000,140.000000
148722,1511299,132,11.000000,12,0.000000,0.000000,66,11.000000,6,-1549.754761,-258.292450,2019-07-22,2019-07-22,2019-07,2019-07,0.000000,NaN
148723,923579,48,4.000000,12,2428.615967,202.384674,0,NaN,0,0.000000,NaN,2019-07-02,2019-07-02,2019-07,2019-07,0.000000,NaN


1.5.3 Average number of unique days between transactions

In [22]:
within_month = date_diff.loc[date_diff.date_diff > 0]
within_month['date_order_month'] = within_month['date_order'].dt.strftime('%Y-%m')
within_month['date_order_shift_month'] = within_month['date_order_shift'].dt.strftime('%Y-%m')
within_month = within_month.loc[within_month.date_order_month == within_month.date_order_shift_month]

<ipython-input-22-825c42446628>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  within_month['date_order_month'] = within_month['date_order'].dt.strftime('%Y-%m')
<ipython-input-22-825c42446628>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  within_month['date_order_shift_month'] = within_month['date_order_shift'].dt.strftime('%Y-%m')


In [23]:
within_month = within_month.groupby(['client_id','date_order_month']).agg({'date_diff':['mean']}).reset_index()
within_month.columns = list(map(''.join, within_month.columns.values))
within_month = within_month.rename(
    columns={"date_diffmean": "Average_Purchase_Day"})

In [24]:
within_month = within_month.loc[within_month.Average_Purchase_Day > 0].groupby(['client_id']).agg({'Average_Purchase_Day':['mean']}).reset_index()
within_month.columns = list(map(''.join, within_month.columns.values))
within_month = within_month.rename(
    columns={"Average_Purchase_Daymean": "Average_Purchase_Day_Within_Month"})

In [25]:
df_customer = df_customer.merge(within_month, on='client_id')
df_customer

,client_id,Total_Quantity,Average_Quantity,Number_of_Purchase,Total_Sales,Average_Sales,R_Total_Quantity,R_Average_Quantity,R_Number_of_Purchase,R_Total_Sales,R_Average_Sales,Last_Activity_Day,First_Activity_Day,Last_Activity_Month,First_Activity_Month,Average_Purchase_Day_All,Average_Purchase_Day,Average_Purchase_Day_Within_Month
0,188502,140138,227.496753,616,66700.335938,108.279770,42,5.250000,8,-1054.278564,-131.784821,2019-09-05,2017-09-25,2019-09,2017-09,1.154472,5.819672,6.009425
1,1977896,293627,567.943907,517,145390.906250,281.220337,260,43.333333,6,-2302.838867,-383.806488,2018-11-05,2017-09-25,2018-11,2017-09,0.786822,4.951220,4.388946
2,2086861,19161,58.956923,325,40332.011719,124.098495,2018,504.500000,4,-1377.861328,-344.465332,2019-09-12,2017-09-25,2019-09,2017-09,2.212963,6.894231,5.566558
3,482822,6740,37.444444,180,16716.583984,92.869911,0,NaN,0,0.000000,NaN,2019-09-06,2017-09-25,2019-09,2017-09,3.972067,16.534884,8.696429
4,224223,27219,26.096836,1043,124999.437500,119.846054,105,8.076923,13,-5282.967285,-406.382111,2019-09-20,2017-09-25,2019-09,2017-09,0.695777,3.625000,4.095230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148720,144085,69,13.800000,5,235.405930,47.081184,21,21.000000,1,-1232.886475,-1232.886475,2019-08-29,2019-08-27,2019-08,2019-08,0.500000,2.000000,2.000000
148721,450600,18,3.000000,6,0.000000,0.000000,3,3.000000,1,-2.290800,-2.290800,2019-08-06,2018-10-30,2019-08,2018-10,56.000000,140.000000,NaN
148722,1511299,132,11.000000,12,0.000000,0.000000,66,11.000000,6,-1549.754761,-258.292450,2019-07-22,2019-07-22,2019-07,2019-07,0.000000,NaN,NaN
148723,923579,48,4.000000,12,2428.615967,202.384674,0,NaN,0,0.000000,NaN,2019-07-02,2019-07-02,2019-07,2019-07,0.000000,NaN,NaN


1.6 Channel Sales

In [26]:
df.order_channel.unique

<bound method Series.unique of 0                 online
1                 online
2               by phone
3               by phone
4           at the store
                ...     
26061256        by phone
26061257        by phone
26061258        by phone
26061259        by phone
26061260        by phone
Name: order_channel, Length: 26061261, dtype: category
Categories (5, object): ['at the store', 'by phone', 'during the visit of a sales rep', 'online',
                         'other']>

In [27]:
df_store = df.loc[df.order_channel == 'at the store']
df_phone = df.loc[df.order_channel == 'by phone']
df_visit = df.loc[df.order_channel == 'during the visit of a sales rep']
df_online = df.loc[df.order_channel == 'online']
df_other = df.loc[df.order_channel == 'other']

In [28]:
store_track = df_store.groupby('client_id').agg({'quantity':['sum','mean'],'sales_net': ['count','sum','mean']}).reset_index()
store_track.columns = list(map(''.join, store_track.columns.values))
store_track = store_track.rename(
    columns={"quantitysum": "Store_Total_Quantity", 
             "sales_netcount": "Store_Number_of_Purchase", 
             "sales_netsum": "Store_Total_Sales", 
             "quantitymean": "Store_Average_Quantity", 
             "sales_netmean": "Store_Average_Sales"})
store_track.head()

,client_id,Store_Total_Quantity,Store_Average_Quantity,Store_Number_of_Purchase,Store_Total_Sales,Store_Average_Sales
0,1000078,3399,82.902439,41,9215.633789,224.771561
1,1000352,17804,51.456647,346,21706.337891,62.735077
2,1000380,0,NaN,0,0.000000,NaN
3,1000618,0,NaN,0,0.000000,NaN
4,1000680,0,NaN,0,0.000000,NaN


In [29]:
phone_track = df_phone.groupby('client_id').agg({'quantity':['sum','mean'],'sales_net': ['count','sum','mean']}).reset_index()
phone_track.columns = list(map(''.join, phone_track.columns.values))
phone_track = phone_track.rename(
    columns={"quantitysum": "Phone_Total_Quantity", 
             "sales_netcount": "Phone_Number_of_Purchase", 
             "sales_netsum": "Phone_Total_Sales", 
             "quantitymean": "Phone_Average_Quantity", 
             "sales_netmean": "Phone_Average_Sales"})
phone_track.head()

,client_id,Phone_Total_Quantity,Phone_Average_Quantity,Phone_Number_of_Purchase,Phone_Total_Sales,Phone_Average_Sales
0,1000078,3534,93.000000,38,21059.140625,554.187927
1,1000352,6892,34.808081,198,51989.273438,262.572083
2,1000380,0,NaN,0,0.000000,NaN
3,1000618,0,NaN,0,0.000000,NaN
4,1000680,0,NaN,0,0.000000,NaN


In [30]:
visit_track = df_visit.groupby('client_id').agg({'quantity':['sum','mean'],'sales_net': ['count','sum','mean']}).reset_index()
visit_track.columns = list(map(''.join, visit_track.columns.values))
visit_track = visit_track.rename(
    columns={"quantitysum": "Visit_Total_Quantity", 
             "sales_netcount": "Visit_Number_of_Purchase", 
             "sales_netsum": "Visit_Total_Sales", 
             "quantitymean": "Visit_Average_Quantity", 
             "sales_netmean": "Visit_Average_Sales"})
visit_track.head()

,client_id,Visit_Total_Quantity,Visit_Average_Quantity,Visit_Number_of_Purchase,Visit_Total_Sales,Visit_Average_Sales
0,1000078,0,NaN,0,0.0,NaN
1,1000352,0,NaN,0,0.0,NaN
2,1000380,0,NaN,0,0.0,NaN
3,1000618,0,NaN,0,0.0,NaN
4,1000680,0,NaN,0,0.0,NaN


In [31]:
online_track = df_online.groupby('client_id').agg({'quantity':['sum','mean'],'sales_net': ['count','sum','mean']}).reset_index()
online_track.columns = list(map(''.join, online_track.columns.values))
online_track = online_track.rename(
    columns={"quantitysum": "Online_Total_Quantity", 
             "sales_netcount": "Online_Number_of_Purchase", 
             "sales_netsum": "Online_Total_Sales", 
             "quantitymean": "Online_Average_Quantity", 
             "sales_netmean": "Online_Average_Sales"})
online_track.head()

,client_id,Online_Total_Quantity,Online_Average_Quantity,Online_Number_of_Purchase,Online_Total_Sales,Online_Average_Sales
0,1000078,6428,84.578947,76,0.000000,0.000000
1,1000352,99,5.823529,17,4625.596191,272.093903
2,1000380,0,NaN,0,0.000000,NaN
3,1000618,0,NaN,0,0.000000,NaN
4,1000680,0,NaN,0,0.000000,NaN


In [32]:
other_track = df_other.groupby('client_id').agg({'quantity':['sum','mean'],'sales_net': ['count','sum','mean']}).reset_index()
other_track.columns = list(map(''.join, other_track.columns.values))
other_track = other_track.rename(
    columns={"quantitysum": "Other_Total_Quantity", 
             "sales_netcount": "Other_Number_of_Purchase", 
             "sales_netsum": "Other_Total_Sales", 
             "quantitymean": "Other_Average_Quantity", 
             "sales_netmean": "Other_Average_Sales"})
other_track.head()

,client_id,Other_Total_Quantity,Other_Average_Quantity,Other_Number_of_Purchase,Other_Total_Sales,Other_Average_Sales
0,1000078,0,NaN,0,0.0,NaN
1,1000352,0,NaN,0,0.0,NaN
2,1000380,0,NaN,0,0.0,NaN
3,1000618,0,NaN,0,0.0,NaN
4,1000680,0,NaN,0,0.0,NaN


In [33]:
df_customer.head()

,client_id,Total_Quantity,Average_Quantity,Number_of_Purchase,Total_Sales,Average_Sales,R_Total_Quantity,R_Average_Quantity,R_Number_of_Purchase,R_Total_Sales,R_Average_Sales,Last_Activity_Day,First_Activity_Day,Last_Activity_Month,First_Activity_Month,Average_Purchase_Day_All,Average_Purchase_Day,Average_Purchase_Day_Within_Month
0,188502,140138,227.496753,616,66700.335938,108.279770,42,5.250000,8,-1054.278564,-131.784821,2019-09-05,2017-09-25,2019-09,2017-09,1.154472,5.819672,6.009425
1,1977896,293627,567.943907,517,145390.906250,281.220337,260,43.333333,6,-2302.838867,-383.806488,2018-11-05,2017-09-25,2018-11,2017-09,0.786822,4.951220,4.388946
2,2086861,19161,58.956923,325,40332.011719,124.098495,2018,504.500000,4,-1377.861328,-344.465332,2019-09-12,2017-09-25,2019-09,2017-09,2.212963,6.894231,5.566558
3,482822,6740,37.444444,180,16716.583984,92.869911,0,NaN,0,0.000000,NaN,2019-09-06,2017-09-25,2019-09,2017-09,3.972067,16.534884,8.696429
4,224223,27219,26.096836,1043,124999.437500,119.846054,105,8.076923,13,-5282.967285,-406.382111,2019-09-20,2017-09-25,2019-09,2017-09,0.695777,3.625000,4.095230


In [34]:
df_customer = df_customer.merge(store_track, on='client_id') 
df_customer = df_customer.merge(phone_track, on='client_id')
df_customer = df_customer.merge(visit_track, on='client_id')
df_customer = df_customer.merge(online_track, on='client_id')
df_customer = df_customer.merge(other_track, on='client_id')
df_customer

,client_id,Total_Quantity,Average_Quantity,Number_of_Purchase,Total_Sales,Average_Sales,R_Total_Quantity,R_Average_Quantity,R_Number_of_Purchase,R_Total_Sales,...,Online_Total_Quantity,Online_Average_Quantity,Online_Number_of_Purchase,Online_Total_Sales,Online_Average_Sales,Other_Total_Quantity,Other_Average_Quantity,Other_Number_of_Purchase,Other_Total_Sales,Other_Average_Sales
0,188502,140138,227.496753,616,66700.335938,108.279770,42,5.250000,8,-1054.278564,...,130582,250.157088,522,50733.84375,97.191269,3,3.0,1,0.1656,0.1656
1,1977896,293627,567.943907,517,145390.906250,281.220337,260,43.333333,6,-2302.838867,...,275805,559.442191,493,135610.68750,275.072388,0,NaN,0,0.0000,NaN
2,2086861,19161,58.956923,325,40332.011719,124.098495,2018,504.500000,4,-1377.861328,...,0,NaN,0,0.00000,NaN,0,NaN,0,0.0000,NaN
3,482822,6740,37.444444,180,16716.583984,92.869911,0,NaN,0,0.000000,...,0,NaN,0,0.00000,NaN,0,NaN,0,0.0000,NaN
4,224223,27219,26.096836,1043,124999.437500,119.846054,105,8.076923,13,-5282.967285,...,0,NaN,0,0.00000,NaN,0,NaN,0,0.0000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148720,144085,69,13.800000,5,235.405930,47.081184,21,21.000000,1,-1232.886475,...,0,NaN,0,0.00000,NaN,0,NaN,0,0.0000,NaN
148721,450600,18,3.000000,6,0.000000,0.000000,3,3.000000,1,-2.290800,...,0,NaN,0,0.00000,NaN,0,NaN,0,0.0000,NaN
148722,1511299,132,11.000000,12,0.000000,0.000000,66,11.000000,6,-1549.754761,...,0,NaN,0,0.00000,NaN,0,NaN,0,0.0000,NaN
148723,923579,48,4.000000,12,2428.615967,202.384674,0,NaN,0,0.000000,...,0,NaN,0,0.00000,NaN,0,NaN,0,0.0000,NaN


In [35]:
df_customer.head()

,client_id,Total_Quantity,Average_Quantity,Number_of_Purchase,Total_Sales,Average_Sales,R_Total_Quantity,R_Average_Quantity,R_Number_of_Purchase,R_Total_Sales,...,Online_Total_Quantity,Online_Average_Quantity,Online_Number_of_Purchase,Online_Total_Sales,Online_Average_Sales,Other_Total_Quantity,Other_Average_Quantity,Other_Number_of_Purchase,Other_Total_Sales,Other_Average_Sales
0,188502,140138,227.496753,616,66700.335938,108.279770,42,5.250000,8,-1054.278564,...,130582,250.157088,522,50733.84375,97.191269,3,3.0,1,0.1656,0.1656
1,1977896,293627,567.943907,517,145390.906250,281.220337,260,43.333333,6,-2302.838867,...,275805,559.442191,493,135610.68750,275.072388,0,NaN,0,0.0000,NaN
2,2086861,19161,58.956923,325,40332.011719,124.098495,2018,504.500000,4,-1377.861328,...,0,NaN,0,0.00000,NaN,0,NaN,0,0.0000,NaN
3,482822,6740,37.444444,180,16716.583984,92.869911,0,NaN,0,0.000000,...,0,NaN,0,0.00000,NaN,0,NaN,0,0.0000,NaN
4,224223,27219,26.096836,1043,124999.437500,119.846054,105,8.076923,13,-5282.967285,...,0,NaN,0,0.00000,NaN,0,NaN,0,0.0000,NaN


1.7 Churn

In [36]:
df_customer['churn_assumption1'] = df_customer.Last_Activity_Month < '2019-06'
df_customer['churn_assumption1']

0         False
1          True
2         False
3         False
4         False
          ...  
148720    False
148721    False
148722    False
148723    False
148724    False
Name: churn_assumption1, Length: 148725, dtype: bool

1.8 RFM

In [37]:
# You have a Text object. The strftime function requires a datetime object.
# The code below takes an intermediate step of converting your Text to a datetime using strptime.

import datetime
testeddate = '2019/09/22'
NOW = datetime.datetime.strptime(testeddate,'%Y/%m/%d')

In [38]:
df_customer["Recency"] = df_customer["Last_Activity_Day"].apply(lambda x: (NOW - x).days)
#df_customer["Frequency"] = df_customer["Total_Quantity"]-df_customer["R_Total_Quantity"]
df_customer["Frequency"] = df_customer["Average_Purchase_Day"]
df_customer["Monetary_Value"] = df_customer["Total_Sales"]+df_customer["R_Total_Sales"]

In [39]:
from datetime import datetime
cleanup = df_customer[['client_id','Average_Purchase_Day','Last_Activity_Day']]
cleanup['Last_Activity_Day'] = pd.to_datetime(df_customer['Last_Activity_Day'], format='%Y-%m-%d')
max_date = datetime.strptime('2019-09-22', '%Y-%m-%d')
cleanup['gap'] = (max_date - cleanup['Last_Activity_Day']).dt.days#.values.astype(int)
cleanup['new_col'] = cleanup['Average_Purchase_Day'].fillna(cleanup['gap'])
df_customer["Frequency"] = cleanup["new_col"]

<ipython-input-39-7c522453055d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleanup['Last_Activity_Day'] = pd.to_datetime(df_customer['Last_Activity_Day'], format='%Y-%m-%d')
<ipython-input-39-7c522453055d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleanup['gap'] = (max_date - cleanup['Last_Activity_Day']).dt.days#.values.astype(int)
<ipython-input-39-7c522453055d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [40]:
#df_customer.to_csv('/content/drive/MyDrive/Elective_Groups/Capstone/Step_5_tttttemp.csv')

In [41]:
R_2 = df_customer["Recency"].quantile(0.2)
R_4 = df_customer["Recency"].quantile(0.4)
R_6 = df_customer["Recency"].quantile(0.6)
R_8 = df_customer["Recency"].quantile(0.8)

def Recency_Quantitle(value):
  if value < R_2:
    return 5
  elif value < R_4:
    return 4
  elif value < R_6:
    return 3
  elif value < R_8:
    return 2
  else:
    return 1

In [42]:
F_2 = df_customer["Frequency"].quantile(0.2)
F_4 = df_customer["Frequency"].quantile(0.4)
F_6 = df_customer["Frequency"].quantile(0.6)
F_8 = df_customer["Frequency"].quantile(0.8)

def Frequency_Quantitle(value):
  if value < F_2:
    return 5
  elif value < F_4:
    return 4
  elif value < F_6:
    return 3
  elif value < F_8:
    return 2
  else:
    return 1

In [43]:
M_2 = df_customer["Monetary_Value"].quantile(0.2)
M_4 = df_customer["Monetary_Value"].quantile(0.4)
M_6 = df_customer["Monetary_Value"].quantile(0.6)
M_8 = df_customer["Monetary_Value"].quantile(0.8)

def Monetary_Value_Quantitle(value):
  if value < M_2:
    return 1
  elif value < M_4:
    return 2
  elif value < M_6:
    return 3
  elif value < M_8:
    return 4
  else:
    return 5

In [44]:
df_customer["Recency_Quantitle"] = df_customer["Recency"].apply(lambda x: Recency_Quantitle(x))
df_customer["Frequency_Quantitle"] = df_customer["Frequency"].apply(lambda x: Frequency_Quantitle(x))
df_customer["Monetary_Value_Quantitle"] = df_customer["Monetary_Value"].apply(lambda x: Monetary_Value_Quantitle(x))

In [45]:
df_customer['RFM'] = df_customer['Recency_Quantitle']*100 + df_customer['Frequency_Quantitle']*10 +df_customer['Monetary_Value_Quantitle']

In [46]:
def Score_Quantitle(value):
    if value in [555, 554, 544, 545, 454, 455, 445]:
        return "Champion"
    elif value in [543, 444, 435, 355, 354, 345, 344, 335]:
        return "Loyal Customer"
    elif value in [553, 551, 552, 541, 542, 533, 532, 531, 452, 451, 442, 441, 431, 453, 433, 432, 423, 353, 352, 351, 342, 341, 333, 323]:
        return "Potential Loyalist"
    elif value in [512, 511, 422, 421, 412, 411, 311]:
        return "New Customer"
    elif value in [525, 524, 523, 522, 521, 515, 514, 513, 425, 424, 413, 414, 415, 315, 314, 313]:
        return "Promising"
    elif value in [535, 534, 443, 434, 343, 334, 325, 324]:
        return "Potential Loyalist"
    elif value in [155, 154, 144, 214,215,115, 114, 113]:
        return "Cannot Lose Them"
    elif value in [331, 321, 312, 221, 213]:
        return "About To Sleep"
    elif value in [255, 254, 245, 244, 253, 252, 243, 242, 235, 234, 225, 224, 153, 152, 145, 143, 142, 135, 134, 133, 125, 124]:
        return "At Risk"
    elif value in [332, 322, 231, 241, 251, 233, 232, 223, 222, 132, 123, 122, 212, 211]:
        return "Hibernating"
    elif value in [111, 112, 121, 131, 141, 151]:
        return "Lost"



In [47]:
df_customer["RFM_Segment"] = df_customer["RFM"].apply(lambda x: Score_Quantitle(x))

2.0 To CSV

In [48]:
df_customer.to_csv('/content/drive/MyDrive/Elective_Groups/Capstone/Step_5_Dashboard.csv')